In [ ]:
from config import load_config
import pandas as pd
import os
os.chdir("/datadisk1/ixk5174/project_repo/Telos")
print(os.getcwd())
cfg = load_config("project_config/config.pkl")
cfg.cov_file = cfg.data_output_dir + "/test_cov.tsv"

cov_df = pd.read_csv(cfg.cov_file, sep="\t")
tss_df = pd.read_csv(cfg.tss_labeled_file)
tes_df = pd.read_csv(cfg.tes_labeled_file)

df = cov_df.merge(tss_df, how="outer", left_on=["tss_chrom", "tss_pos"], right_on=["chrom", "position"], indicator=True)

df.head(df[df["_merge"] == "both"].shape[0])
left_df = df[df["_merge"] == "left_only"]
right_df = df[df["_merge"] == "right_only"]
print("left_df.shape", left_df.shape)
print("right_df.shape", right_df.shape)


/datadisk1/ixk5174/project_repo/Telos
left_df.shape (0, 30)
rigth_df.shape (0, 30)
